## Google Colab Rust Setup

The following cell is used to set up and spin up a Jupyter Notebook environment with a Rust kernel using Nix and IPC Proxy.

In [ ]:
!wget -qO- https://gist.github.com/wiseaidev/2af6bef753d48565d11bcd478728c979/archive/3f6df40db09f3517ade41997b541b81f0976c12e.tar.gz | tar xvz --strip-components=1
!bash setup_evcxr_kernel.sh

## 1. Async Traits
### A. Rust Version < v1.75.0 (e.g. 1.74.0)

In [2]:
:dep tokio = { version = "1.35.1", features = ["full"] }

In [3]:
:dep futures = { version = "0.3.30" }

In [4]:
:dep async-trait = { version = "0.1.77" }

#### 1.1 Using `async-trait`

In [5]:
use async_trait::async_trait;

// Define an async trait
#[async_trait]
trait MyAsyncTrait {
    async fn async_method(&self) -> i32;
}

// Implement the async trait
struct MyStruct;

#[async_trait]
impl MyAsyncTrait for MyStruct {
    async fn async_method(&self) -> i32 {
        // Asynchronous logic goes here
        42
    }
}

let my_struct = MyStruct;

let result = my_struct.async_method().await;

result

42

#### 1.12 Using Futures

In [9]:
use std::pin::Pin
use futures::Future;

// Define an async trait
trait MyAsyncTrait {
    fn async_method(&self) -> Pin<Box<dyn Future<Output = i32>>>;
}

// Implement the async trait
struct MyStruct;

impl MyAsyncTrait for MyStruct {
    fn async_method(&self) -> Pin<Box<dyn Future<Output = i32>>> {
        Box::pin(async { 42 })
    }
}


let my_struct = MyStruct;

let result = my_struct.async_method().await;

result

Result: 42


()

### B. Rust v1.75.0

In [6]:
// Define an async trait
trait MyAsyncTrait {
    async fn method(&self) -> i32;
}

// Implement the async trait
struct MyStruct;

impl MyAsyncTrait for MyStruct {
    async fn method(&self) -> i32 {
        42
    }
}

let my_struct = MyStruct;

let result = my_struct.method().await;

result

42

Prior to this version, e.g. **1.74.0**, the compiler would complain about the following:

```sh
error[E0706]: functions in traits cannot be declared `async`
  |
3 |     async fn method(&self) -> i32;
  |     -----^^^^^^^^^^^^^^^^^^^^^^^^^
  |     |
  |     `async` because of this
  |
  = note: `async` trait functions are not currently supported
  = note: consider using the `async-trait` crate: https://crates.io/crates/async-trait
  = note: see issue #91611 <https://github.com/rust-lang/rust/issues/91611> for more information

error[E0706]: functions in traits cannot be declared `async`
   |
10 |     async fn method(&self) -> i32 {
   |     -----^^^^^^^^^^^^^^^^^^^^^^^^
   |     |
   |     `async` because of this
   |
   = note: `async` trait functions are not currently supported
   = note: consider using the `async-trait` crate: https://crates.io/crates/async-trait
   = note: see issue #91611 <https://github.com/rust-lang/rust/issues/91611> for more information

For more information about this error, try `rustc --explain E0706`.
error: could not compile `file` (bin "file") due to 2 previous errors
```

---

## 2. Mutable References In const functions
### A. Rust Version < v1.75.0 (e.g. 1.74.0)

You would need to remove the `const` context or use `static` methods that don't involve mutable references.

In [8]:
fn get_some_fn() -> fn(&mut String) {
    String::clear
}

fn use_const_fn(_f: fn(&mut String)) {
    ()
}

fn some_const_fn() {
    let _f: fn(&mut String) = String::clear;
}

let my_function = get_some_fn();
use_const_fn(my_function);
some_const_fn();

### B. Rust v1.75.0

In [7]:
const fn use_const_fn() {
    let _f: fn(&mut String) = String::clear;
}

use_const_fn();

Prior to this version, e.g. **1.74.0**, the compiler would complain about the following:

```sh
error[E0658]: mutable references are not allowed in constant functions
  |
2 |     let _f: fn(&mut String) = String::clear;
  |         ^^
  |
  = note: see issue #57349 <https://github.com/rust-lang/rust/issues/57349> for more information

For more information about this error, try `rustc --explain E0658`.
error: could not compile `file` (bin "file") due to previous error
```

---

## 3. Exhaustiveness Checking for Half-Open Ranges
### A. Rust Version < v1.75.0 (e.g. 1.74.0)

You would need to add a wildcard `_` to match exhaustively

In [9]:
fn check_range(value: usize) {
    match value {
        ..=42 => println!("Value is 42 or smaller!"),
        43.. => println!("Value is greater than 42"),
        _ => println!("Anything else"),
    }
}

let value = 42;

check_range(value);

Value is 42 or smaller!


### B. Rust v1.75.0

In [ ]:
fn check_range(value: usize) {
    match value {
        ..=42 => println!("Value is 42 or smaller!"),
        43.. => println!("Value is greater than 42"),
    }
}

let value = 42;

check_range(value);
// It complains about it, not sure why. Let's run it with cargo.

In [5]:
use std::process::{Command, Output, Stdio};

// A helper function to execute a shell command from a Rust script
fn execute_command(command: &str) -> Result<(), std::io::Error> {
    let status = Command::new("bash")
        .arg("-c")
        .arg(command)
        .stderr(Stdio::inherit())
        .status()?;

    if status.success() {
        Ok(())
    } else {
        Err(std::io::Error::from_raw_os_error(status.code().unwrap_or(1)))
    }
}

let command = "cd half-open-ranges && cargo +1.75.0 run";

if let Err(err) = execute_command(command) {
    eprintln!("Error executing command: {}", err);
}

    Finished dev [unoptimized + debuginfo] target(s) in 0.00s
     Running `target/debug/half-open-ranges`


Value is 42 or smaller!


()

---

## 4. `char` has the same size and alignment as `u32` on all platforms.

The following was not guaranteed on all platforms.

In [11]:
use std::alloc::Layout;
use std::mem;

// Assuming a platform where char is 4 bytes (UTF-32)
assert_eq!(Layout::new::<char>(), Layout::new::<u32>());

// However, when dealing with UTF-16 encoding, char could be 2 bytes
assert_ne!(Layout::new::<char>(), Layout::new::<u16>());

assert_eq!(mem::size_of::<char>(), mem::size_of::<u32>());

println!("Size of char: {} bytes", mem::size_of::<char>());
println!("Size of u32: {} bytes", mem::size_of::<u32>());

Size of char: 4 bytes
Size of u32: 4 bytes


---

## 5. Allow partially moved values in a `match` statement
### A. Rust Version < v1.75.0 (e.g. 1.74.0)

The following was not allowed in versions prior to 1.75.0

In [16]:
let foo = (Box::new(27), Box::new(42));
let x = foo.0;
let y = foo.1;

match foo {
    (_, _) => {
        println!("The sun of x and y is: {:?}", *x + *y);
    }
}

Error: use of partially moved value: `foo`

### B. Rust v1.75.0

In [20]:
let command = "cd partially-moved-values && cargo +1.75.0 run";

if let Err(err) = execute_command(command) {
    eprintln!("Error executing command: {}", err);
}

    Finished dev [unoptimized + debuginfo] target(s) in 0.01s
     Running `target/debug/partially-moved-values`


The sun of x and y is: 69


()

Here is a simple memory layout of what is happening under the hood.
![memory layout](https://pbs.twimg.com/media/GDbtB0kXgAA6MRE?format=jpg&name=medium)

---

## 6. add notes about non-compliant FP behavior
Just documentation about non-compliant FP behavior on 32bit x86 targets.

---

## 7. Stabilize Ratified RISC-V Target Features
Rust Stabilization on RISC-V Based IoT devices.

---

## 8. Rework negative coherence to properly consider impls that only partly overlap.
Compiler shenanigans related to partially overlapping impls you don't need to learn about it.

---

## 9. Bump `COINDUCTIVE_OVERLAP_IN_COHERENCE` to deny, and warn in dependencies.
### A. Rust Version < v1.75.0 (e.g. 1.74.0)

This feature is related to the coherence rules for trait implementations. Specifically, it addresses the potential future breakage when implementing `PartialEq<Interval<_>>` for `Interval<_>`.

In [ ]:
use std::cmp::Ordering;
use std::marker::PhantomData;

#[derive(PartialEq, Default)]
pub(crate) struct Interval<T>(PhantomData<T>);

impl<T, Q> PartialEq<Q> for Interval<T>
where
    Q: PartialOrd,
{
    fn eq(&self, _: &Q) -> bool {
        // For simplicity, let's assume equality if the underlying type T is comparable to Q
        true
    }
}

impl<T, Q> PartialOrd<Q> for Interval<T>
where
    Q: PartialOrd,
{
    fn partial_cmp(&self, _: &Q) -> Option<Ordering> {
        // For simplicity, let's assume the ordering is always equal for PhantomData<T>
        Some(Ordering::Equal)
    }
}

let interval1 = Interval::<u32>::default();
let interval2 = Interval::<u32>::default();

if interval1 == interval2 {
    println!("Intervals are equal!");
}

In this example, we have a `PartialEq` implementation for `Interval<T>` where `T` can be borrowed as `Q`, and `Q` must be `PartialOrd`. The lint is warning that this implementation might cause issues in the future, especially if Rust decides to make `Interval<_>: PartialOrd` hold in future releases, causing the implementations to overlap.

This would cause a warning in 1.74.0:

In [37]:
let command = "cd coinductive_overlap_in_coherence && cargo +1.74.0 run";

if let Err(err) = execute_command(command) {
    eprintln!("Error executing command: {}", err);
}

   Compiling coinductive_overlap_in_coherence v0.1.0 (/home/mahmoud/Desktop/Rust/rust-releases/1.75/coinductive_overlap_in_coherence)
 --> src/main.rs:7:1
  |
4 | #[derive(PartialEq, Default)]
  |          --------- the second impl is here
...
7 | impl<T, Q> PartialEq<Q> for Interval<T>
  | ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^ the first impl is here
  |
  = warning: this was previously accepted by the compiler but is being phased out; it will become a hard error in a future release!
  = note: for more information, see issue #114040 <https://github.com/rust-lang/rust/issues/114040>
  = note: impls that are not considered to overlap may be considered to overlap in the future
  = note: `Interval<_>: PartialOrd` may be considered to hold in future releases, causing the impls to overlap
  = note: `#[warn(coinductive_overlap_in_coherence)]` on by default

    Finished dev [unoptimized + debuginfo] target(s) in 0.27s
     Running `target/debug/coinductive_overlap_in_coherence`


Intervals are equal!


()

### B. Rust v1.75.0
The same program would cause an error in 1.75.0

In [38]:
use std::cmp::Ordering;
use std::marker::PhantomData;

#[derive(PartialEq, Default)]
pub(crate) struct Interval<T>(PhantomData<T>);

impl<T, Q> PartialEq<Q> for Interval<T>
where
    Q: PartialOrd,
{
    fn eq(&self, _: &Q) -> bool {
        // For simplicity, let's assume equality if the underlying type T is comparable to Q
        true
    }
}

impl<T, Q> PartialOrd<Q> for Interval<T>
where
    Q: PartialOrd,
{
    fn partial_cmp(&self, _: &Q) -> Option<Ordering> {
        // For simplicity, let's assume the ordering is always equal for PhantomData<T>
        Some(Ordering::Equal)
    }
}

let interval1 = Interval::<u32>::default();
let interval2 = Interval::<u32>::default();

if interval1 == interval2 {
    println!("Intervals are equal!");
}

Error: implementations of `PartialEq<Interval<_>>` for `Interval<_>` will conflict in the future

Error: unused variable: `other`

---

## 10. Consider alias bounds when computing liveness in NLL.
### A. Rust Version < v1.75.0 (e.g. 1.74.0)

This feature addresses improvements in the Non-Lexical Lifetimes (NLL) analysis related to liveness computation. The goal is to make the liveness analysis more accurate, particularly in the presence of aliases, such as opaque types, and to ensure soundness in the computation.

In [11]:
// Example trait with a region parameter
trait Captures<'a> {}
impl<T> Captures<'_> for T {}

// Opaque function returning an impl Trait with region capture
fn capture<'o>(_: &'o mut ()) -> impl Sized + Captures<'o> + 'static {
    // Implementation details
    todo!()
}

// Function using the opaque type with a borrow checker issue
fn test_two_mut(mut x: ()) {
    let _f1 = capture(&mut x);
    let _f2 = capture(&mut x);
    // In the fixed version, the liveness analysis considers 'static bound
    // and avoids borrow checker errors in certain cases.
    //~^ ERROR cannot borrow `x` as mutable more than once at a time
}

test_two_mut(());

Error: cannot borrow `x` as mutable more than once at a time

In this example, `test_two_mut` calls the `capture` function twice, capturing a mutable reference to `x` both times which would result in an error.

### B. Rust v1.75.0
This would raise the following exception:

In [10]:
let command = "cd alias-bounds && cargo +1.75.0 run";

if let Err(err) = execute_command(command) {
    eprintln!("Error executing command: {}", err);
}

    Finished dev [unoptimized + debuginfo] target(s) in 0.05s
     Running `target/debug/alias-bounds`
thread 'main' panicked at src/main.rs:8:5:
not yet implemented
stack backtrace:
   0: rust_begin_unwind
             at /rustc/82e1608dfa6e0b5569232559e3d385fea5a93112/library/std/src/panicking.rs:645:5
   1: core::panicking::panic_fmt
             at /rustc/82e1608dfa6e0b5569232559e3d385fea5a93112/library/core/src/panicking.rs:72:14
   2: core::panicking::panic
             at /rustc/82e1608dfa6e0b5569232559e3d385fea5a93112/library/core/src/panicking.rs:127:5
   3: alias_bounds::capture
             at ./src/main.rs:8:5
   4: alias_bounds::test_two_mut
             at ./src/main.rs:13:15
   5: alias_bounds::main
             at ./src/main.rs:21:5
   6: core::ops::function::FnOnce::call_once
             at /rustc/82e1608dfa6e0b5569232559e3d385fea5a93112/library/core/src/ops/function.rs:250:5
note: Some details are omitted, run with `RUST_BACKTRACE=full` for a verbose backtrace.
Error

()

---

## 11. Add the V (vector) extension to the riscv64-linux-android target spec.
RISC-V related enhancements.

---

## 12. Automatically enable cross-crate inlining for small functions.
### A. Rust Version < v1.75.0 (e.g. 1.74.0)
In versions prior to 1.75.0, we often add `#[inline]` manually to hint the compiler for inlining optimizations for small functions:

In [13]:
#[inline]
fn add(a: i32, b: i32) -> i32 {
    a + b
}

let result = add(3, 5);
result

8

### B. Rust v1.75.0
Implicit `#[inline]`:

In [14]:
fn add(a: i32, b: i32) -> i32 {
    a + b
}

let result = add(3, 5);
result

8

---

## 13. Added several new tier 3 targets:
- `csky-unknown-linux-gnuabiv2hf`
- `i586-unknown-netbsd`
- `mipsel-unknown-netbsd`

---

## 14. Implement `Default` for ExitCode.
### A. Rust Version < v1.75.0 (e.g. 1.74.0)

In [25]:
let command = "cd default-exitcode && cargo +1.74.0 run";

if let Err(err) = execute_command(command) {
    eprintln!("Error executing command: {}", err);
}

   Compiling default-exitcode v0.1.0 (/home/mahmoud/Desktop/Rust/rust-releases/1.75/default-exitcode)
error[E0277]: the trait bound `ExitCode: Default` is not satisfied
  --> src/main.rs:12:41
   |
12 |     let exit_code: ExitCode = cmd_ops().unwrap_or_default();
   |                                         ^^^^^^^^^^^^^^^^^ the trait `Default` is not implemented for `ExitCode`
   |
note: required by a bound in `Result::<T, E>::unwrap_or_default`
  --> /rustc/79e9716c980570bfd1f666e3b16ac583f0168962/library/core/src/result.rs:1108:5

For more information about this error, try `rustc --explain E0277`.
error: could not compile `default-exitcode` (bin "default-exitcode") due to previous error
Error executing command: Network is unreachable (os error 101)


()

### B. Rust v1.75.0

In [22]:
use std::process::ExitCode;

fn cmd_ops() -> Result<ExitCode, ()> {
    if true {
        Ok(ExitCode::SUCCESS)
    } else {
        Err(())
    }
}

fn main() -> Result<ExitCode, ()> {
    let exit_code: ExitCode = cmd_ops().unwrap_or_default();

    Ok(exit_code)
}

main()

Ok(ExitCode(unix_exit_status(0)))

---

## 15. Implement `Not`, `Bit{And,Or}{,Assign}` for IP addresses.
### A. Rust Version < v1.75.0 (e.g. 1.74.0)

In [27]:
use std::net::Ipv4Addr;
use std::ops::{BitOr, Not};

#[derive(Clone, Copy)]
struct MyIpv4Addr(Ipv4Addr);

impl Not for MyIpv4Addr {
    type Output = MyIpv4Addr;

    fn not(self) -> MyIpv4Addr {
        MyIpv4Addr(Ipv4Addr::new(
            !self.0.octets()[0],
            !self.0.octets()[1],
            !self.0.octets()[2],
            !self.0.octets()[3],
        ))
    }
}

impl BitOr<MyIpv4Addr> for MyIpv4Addr {
    type Output = MyIpv4Addr;

    fn bitor(self, other: MyIpv4Addr) -> MyIpv4Addr {
        MyIpv4Addr(Ipv4Addr::new(
            self.0.octets()[0] | other.0.octets()[0],
            self.0.octets()[1] | other.0.octets()[1],
            self.0.octets()[2] | other.0.octets()[2],
            self.0.octets()[3] | other.0.octets()[3],
        ))
    }
}

let ip1 = MyIpv4Addr(Ipv4Addr::new(0, 0, 0, 0));
let ip2 = MyIpv4Addr(Ipv4Addr::new(1, 1, 1, 1));

let result_or = ip1 | ip2;
let result_not = !ip1;

println!("{:?} | {:?} = {:?}", ip1.0, ip2.0, result_or.0);
println!("!{:?} = {:?}", ip1.0, result_not.0);

0.0.0.0 | 1.1.1.1 = 1.1.1.1
!0.0.0.0 = 255.255.255.255


### B. Rust v1.75.0

In [28]:
use std::net::Ipv4Addr;

let ip1 = Ipv4Addr::new(0, 0, 0, 0);
let ip2 = Ipv4Addr::new(1, 1, 1, 1);

let result_or = ip1 | ip2;
let result_not = !&ip1;

println!("{:?} | {:?} = {:?}", ip1, ip2, result_or);
println!("!{:?} = {:?}", ip1, result_not);

0.0.0.0 | 1.1.1.1 = 1.1.1.1
!0.0.0.0 = 255.255.255.255


### TODO

### Source
[github.com/rust-lang/rust/RELEASES](https://github.com/rust-lang/rust/blob/master/RELEASES.md#version-1750-2023-12-28)

---
---